In [ ]:
!pip install -qq -U mlxtend

     |████████████████████████████████| 1.3 MB 8.4 MB/s 


In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from tqdm.auto import tqdm
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
STOPWORDS = set(nltk.corpus.stopwords.words("english") + ["says", "us", "said" ,"new", "optout", "stop", "ask", "stay", "talk","communityrepli", "free","activ", "agenc"])
PORTER_STEMMER = PorterStemmer()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# reuters_4000 = pd.read_csv("/content/drive/MyDrive/Atharva_Scrape/Reuters_Health-5000.csv")
reuters_4000 = pd.read_csv('/content/drive/MyDrive/Project 2/Colab/Atharva_Scrape/Reuters_Health-5000.csv')

In [ ]:
len(reuters_4000)

5416

In [ ]:
reuters_4000.iloc[0]['tweet']

"Walgreens Boots Alliance Inc kept its 2022 earnings forecast unchanged even as it beat estimates for second-quarter results, sending the drugstore chain's shares down 7% on fears of slower-than-expected growth for the rest of the year. https://t.co/ehuvVMCxWG"

In [ ]:
def clean_tweets(x):
    # Lowercase
    sentence = x.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub('http[s]?://\S+', '', sentence)
    sentence = re.sub(r'([^a-zA-Z ]+?)', '', sentence)
    #print(sentence)
    #sentence = sentence.replace('\n', '')
    # Remove URLs
    
    #print(sentence)
    # Remove double spacing
    #sentence = re.sub('\s+', ' ', sentence)
    tokenized_tweet = [word for word in word_tokenize(sentence) if word not in STOPWORDS]
    tokenized_tweet = [PORTER_STEMMER.stem(word) for word in tokenized_tweet]
    return tokenized_tweet

In [ ]:
tqdm.pandas()
reuters_4000["tokenized_tweet"] = reuters_4000["tweet"].progress_apply(lambda x : clean_tweets(x))

  0%|          | 0/5416 [00:00<?, ?it/s]

In [ ]:
tokenized_tweets = reuters_4000["tokenized_tweet"].values.tolist()
te = TransactionEncoder()
te_ary = te.fit(tokenized_tweets).transform(tokenized_tweets)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

In [ ]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [ ]:
type(frequent_itemsets)

pandas.core.frame.DataFrame

In [ ]:
len(frequent_itemsets)

18358

In [ ]:
frequent_itemsets[frequent_itemsets['length'] >= 2].head(30)

support              itemsets  length
344  0.020126       (accord, covid)       2
345  0.015694     (covid, administ)       2
346  0.011263      (dose, administ)       2
347  0.014771    (vaccin, administ)       2
348  0.014586   (administr, approv)       2
349  0.012925   (administr, author)       2
350  0.026034    (covid, administr)       2
351  0.041728     (administr, drug)       2
352  0.040990     (administr, food)       2
353  0.010340   (administr, friday)       2
354  0.016617      (administr, inc)       2
355  0.010155   (monday, administr)       2
356  0.011078  (administr, tuesday)       2
357  0.017171   (administr, vaccin)       2
358  0.014771        (covid, adult)       2
359  0.010524         (adult, drug)       2
360  0.023449       (covid, africa)       2
361  0.022895       (africa, south)       2
362  0.019572      (africa, vaccin)       2
363  0.014402     (variant, africa)       2
364  0.011448         (age, author)       2
365  0.024003       (age, children)       2
366  0.039697          (covid, age)       2
367  0.013848           (age, drug)       2
368  0.017356            (age, inc)       2
369  0.015140         (age, pfizer)       2
370  0.011263        (age, tuesday)       2
371  0.036558         (age, vaccin)       2
372  0.015879           (age, year)       2
373  0.026403        (covid, agenc)       2

In [ ]:
frequent_itemsets.to_csv('/content/drive/MyDrive/Project 2/Colab/Atharva_Scrape/arm_reuters_health.csv')